In [1]:
import sqlite3 as sq
import pandas as pd

In [2]:
con = sq.connect("NPOPlayer.db")
def run_query(con, table, query):
    cur = con.cursor()
    cur.execute(query)
    results = cur.fetchall()
    return results

def convert_db_table_to_DF(con, table):
    header_query = "SELECT name FROM pragma_table_info('" + table + "') ORDER BY cid;"
    cols_init = run_query(con, table, header_query)
    cols = [cols_init[i][0] for i in range(len(cols_init))]
    content_query = "SELECT * from " + table
    data = run_query(con, table, content_query)
    df = pd.DataFrame(data, columns= cols)
    return df

In [4]:
df = convert_db_table_to_DF(con, "NPOPLAYER")

In [20]:
import spacy
from tqdm import tqdm

# use displacy to visually show the entities. 
from spacy import displacy

# load spacy model. Alternatively you can use en_core_web_lg
nlp = spacy.load("nl_core_news_sm")

In [32]:
# df_sub = df.copy()
df_sub = df.sample(6000).copy()

In [33]:
def process(x):
  # there are some pesky NaN in the data. Easy but not so elegant way to fix this.
  if pd.isna(x) == False:
    doc = nlp(x)
  else:
    doc = ''
  return doc


tqdm.pandas()
df_sub['plot_entities'] = df_sub['longSummary'].progress_apply(lambda x: process(x))


100%|██████████| 6000/6000 [00:52<00:00, 114.66it/s]


In [83]:
def get_events(x):
  events = []
  if x != '':
    for entities in x.ents:
      if entities.label_ == 'EVENT':
        events.append(entities.text)
      elif entities.label_ == 'LAW':
        events.append(entities.text)
      elif entities.label_ == 'LOC':
        events.append(entities.text)
      elif entities.label_ == 'PERSON':
        events.append(entities.text)
      elif entities.label_ == 'PRODUCT':
        events.append(entities.text)
      elif entities.label_ == 'WORK_OF_ART':
        events.append(entities.text)
  return events

# code goes here
df_sub['tags'] = df_sub['plot_entities'].progress_apply(lambda x: get_events(x))
df_sub[df_sub["tags"].str.len() == 0]

100%|██████████| 6000/6000 [00:00<00:00, 39214.19it/s]


,mediaID,mainTitle,broadcaster,longSummary,shortSummary,icons,subTitle,thumbnail,kids,shareLink,plot_entities,events,tags
51520,VPRO_1127899,Holland Festival,VPRO,Opgave volgt.,Opgave volgt.,,14 jun 2008 17:25 - Seizoen 1 Afl. 3 - Holland...,https://images.poms.omroep.nl/image/s2560/c256...,false,https://www.npostart.nl/VPRO_1127899,"(Opgave, volgt, .)",[],[]
71056,AT_2077369,Nederland Verhuist,AVROTROS,"Daan en Barabara, een jong stel uit Leiden, ga...","Daan en Barabara, een jong stel uit Leiden, ga...",,27 apr 2017 19:50 - Seizoen 10 Afl. 12 - Neder...,https://images.npo.nl/header/2560x1440/893136.jpg,false,https://www.npostart.nl/AT_2077369,"(Daan, en, Barabara, ,, een, jong, stel, uit, ...",[],[]
81684,POW_00263825,NOS Jeugdjournaal,NOS,,,,28 dec 2010 18:45 - Seizoen 13 Afl. 304 - NOS ...,https://images.npo.nl/header/2560x1440/333970.jpg,false,https://www.npostart.nl/POW_00263825,(),[],[]
3548,NPS_1237999,Andere Tijden,VPRO,,,,20 mrt 2014 21:25 - Seizoen 24 Afl. 11 - Waken...,https://images.npo.nl/header/2560x1440/549850.jpg,false,https://www.npostart.nl/NPS_1237999,(),[],[]
100974,POW_00325825,PowNews Flits,PowNed,Latenight nieuws.,Latenight nieuws.,,11 nov 2010 22:50 - Seizoen 2 Afl. 41 - PowNews,https://images.npo.nl/header/2560x1440/345055.jpg,false,https://www.npostart.nl/POW_00325825,"(Latenight, nieuws, .)",[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67934,POW_00791070,Mike de Ridder,KRO,De moeder van Mike is druk met de voorbereidin...,De moeder van Mike is druk met de voorbereidin...,leeftijd-al,24 sep 2014 12:06 - Seizoen 2 Afl. 14 - Mike d...,https://images.npo.nl/header/2560x1440/538845.jpg,false,https://www.npostart.nl/POW_00791070,"(De, moeder, van, Mike, is, druk, met, de, voo...",[],[]
3304,VPWON_1236044,Andere Tijden,NTR,Ze dachten dat de oorlog over was. Even na Dol...,Ze dachten dat de oorlog over was. Even na Dol...,,7 apr 2015 21:20 - Seizoen 27 Afl. 12 - Verra...,https://images.npo.nl/header/2560x1440/779384.jpg,false,https://www.npostart.nl/VPWON_1236044,"(Ze, dachten, dat, de, oorlog, over, was, ., E...",[],[]
58623,VPWON_1272823,Koek & Ei,EO,Ga mee met Otto naar de wigwam. Welke rare sni...,Ga mee met Otto naar de wigwam. Welke rare sni...,,21 mrt 2017 12:15 - Seizoen 11 Afl. 2 - De wig...,https://images.npo.nl/header/2560x1440/878869.jpg,false,https://www.npostart.nl/VPWON_1272823,"(Ga, mee, met, Otto, naar, de, wigwam, ., Welk...",[],[]
74451,POW_03140682,NOS EK Voetbal,NOS,Wedstrijdanalyse van het EK-duel.,Wedstrijdanalyse van het EK-duel.,,NOS EK Voetbal: Turkije - Kroatië,https://images.npo.nl/header/2560x1440/759531.jpg,false,https://www.npostart.nl/POW_03140682,"(Wedstrijdanalyse, van, het, EK-duel, .)",[],[]
